In [ ]:
import os
# you may need to install selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import urllib.request
from bs4 import BeautifulSoup
import moviepy.editor as mp
import speech_recognition as sr
import time

import pandas as pd

### Download chromedriver and save the .exe file into your folder

chromedriver.exe (Download from: https://chromedriver.chromium.org/)

In [ ]:
df = pd.read_excel("data/videos_all.xlsx")

In [ ]:
driver = webdriver.Chrome(r'/Users/Flame/Dropbox/Summer/QMB6930/chromedriver') # local path of chromedriver file
# A new chrome window will pop out

# Set directory
os.chdir('/Users/Flame/Dropbox/Summer/QMB6930') # your folder path

# Create a temp directory
if not os.path.exists("./temp"):
    os.makedirs("./temp")

In [ ]:
result_l = []

# this code may run for hours
i = 0
for url in df['video_url']:
    
    i += 1
    driver.get(url)
    
    # give chrome some time to load the page
    time.sleep(2)
    
    # 1. get page_source
    # 2. Parse HTML
    # 3. Find the node for the video
    video_url = BeautifulSoup(driver.page_source, 'html.parser').find("video")
    
    try:
        # Download video and save as "./temp/temp.mp4"
        urllib.request.urlretrieve(video_url["src"], './temp/temp.mp4')
        
        # Rip audio from the video
        clip = mp.VideoFileClip('./temp/temp.mp4')
        clip.audio.write_audiofile('./temp/temp.wav')
        
        # Run Speech Recognition
        r = sr.Recognizer()
        audio_file = sr.AudioFile('./temp/temp.wav')
        
        with audio_file as source:
            r.adjust_for_ambient_noise(source)
            audio = r.record(source)
            
        result = r.recognize_google(audio)

    except:
        result = ''
        
    result_l.append(result)
    print('video {} is done.'.format(str(i)))

In [ ]:
# close the driver
driver.quit()

In [ ]:
# make sure the outcome is True
len(result_l) == len(df['video_url'])

In [ ]:
# add the transcripts to a new column
df['video_transcript'] = result_l

# change video id and timestamp to str type
df = df.astype({"video_id": str, 'video_timestamp': str})

In [ ]:
# save the new dataframe as an excel file (replace the original one)
df.to_excel("videos_all.xlsx",sheet_name='videos', index=False) 